In [0]:
%fs ls /mnt/g5/landingzone/

In [0]:
# Create a Database

In [0]:
import re
userName = spark.sql("SELECT CURRENT_USER").collect()[0]['current_user()']
userName0 = userName.split("@")[0]
userName0 = re.sub('[!#$%&\'*+-/=?^`{}|\.]+', '_', userName0)
userName1 = userName.split("@")[1]
userName = f'{userName0}@{userName1}'
dbutils.fs.mkdirs(f"/Users/{userName}/data")
userDir = f"/Users/{userName}/data"
databaseName = f"{userName0}_FinalProject_01"

print('databaseName ' + databaseName)
print('UserDir ' + userDir)

spark.sql(f"CREATE DATABASE IF NOT EXISTS {databaseName}")
spark.sql(f"use {databaseName}")

In [0]:
#Create a AutoLoader to load the files from the path

In [0]:
print('UserDir ' + userDir)

In [0]:
rawDataSource='/mnt/g5/landingzone'
bronzeCheckpoint = f"{userDir}/bronze_check_point"
bronzeTable = f"{userDir}/bronze"
bronzeSchema = f"{userDir}/bronze_schema"
bronze_delta_path='/mnt/g5/bronze/'

In [0]:
%fs ls /Users/veg940@g.harvard.edu/data/bronze_schema

## Load Data from Landing Zone to Bronze Delta Path

In [0]:
import os
from pyspark.sql.functions import input_file_name,expr
from pyspark.sql.types import ArrayType,IntegerType,StringType

**Store_Sales Data Load in to Bronze Layer**

In [0]:
schema_store_sales = spark.read.format("csv").option("inferSchema", True).option("header", "true").load(f"{rawDataSource}/store_sales.csv").schema

In [0]:
schema_store_sales

In [0]:
df = spark.readStream.format("cloudFiles") \
  .option("cloudFiles.format", "csv") \
  .option("pathGlobFilter", "*store_sales*.csv") \
  .option("header", True) \
  .schema(schema_store_sales) \
  .load(f"{rawDataSource}") \
  #.withColumn("filename",input_file_name())

In [0]:
display(df)

In [0]:
stores_sales_checkpoint_location = f"{bronzeCheckpoint}/stores_sales"
bronze_stores_sales_deltapath='/mnt/g5/bronze/stores_sales'
dbutils.fs.rm(stores_sales_checkpoint_location, True) #reset checkpoint so it reloads the file
df.writeStream.option("path", bronze_stores_sales_deltapath).outputMode("append").format("delta").option("checkpointLocation", stores_sales_checkpoint_location).option("mergeSchema", "true").table("stores_sales")

In [0]:
%sql
SELECT * FROM stores_sales

**Oil Prices Data Load in to Bronze Layer**

In [0]:
schema_oilprices = spark.read.format("csv").option("inferSchema", True).option("header", "true").load(f"{rawDataSource}/oil_prices.csv").schema

In [0]:
oil_prices_df = spark.readStream.format("cloudFiles") \
  .option("cloudFiles.format", "csv") \
  .option("pathGlobFilter", "*oil_prices*.csv") \
  .option("header", True) \
  .schema(schema_oilprices) \
  .load(f"{rawDataSource}") \
  #.withColumn("filename",input_file_name())

In [0]:
oil_prices_checkpoint_location = f"{bronzeCheckpoint}/oil_prices"
bronze_oil_prices_deltapath='/mnt/g5/bronze/oil_prices'
dbutils.fs.rm(oil_prices_checkpoint_location, True) #reset checkpoint so it reloads the file
oil_prices_df.writeStream.option("path", bronze_oil_prices_deltapath).outputMode("append").format("delta").option("checkpointLocation", oil_prices_checkpoint_location).option("mergeSchema", "true").table("oil_prices")

In [0]:
display(oil_prices_df)

In [0]:
%sql
SELECT * FROM oil_prices

**Store Master Data Load in to Bronze Layer**

In [0]:
schema_store_master = spark.read.format("csv").option("inferSchema", True).option("header", "true").load(f"{rawDataSource}/store_master.csv").schema

In [0]:
store_master_df = spark.readStream.format("cloudFiles") \
  .option("cloudFiles.format", "csv") \
  .option("pathGlobFilter", "*store_master*.csv") \
  .option("header", True) \
  .schema(schema_store_master) \
  .load(f"{rawDataSource}") \
  #.withColumn("filename",input_file_name())

In [0]:
display(store_master_df)

In [0]:
store_master_checkpoint_location = f"{bronzeCheckpoint}/store_master"
bronze_store_master_deltapath='/mnt/g5/bronze/store_master'
if dbutils.fs.rm(bronze_store_master_deltapath, True):
  print(f"{bronze_store_master_deltapath} directory deleted...")
dbutils.fs.rm(store_master_checkpoint_location, True) #reset checkpoint so it reloads the file
store_master_df.writeStream.option("path", bronze_store_master_deltapath).outputMode("append").format("delta").option("checkpointLocation", store_master_checkpoint_location).option("mergeSchema", "true").table("store_master")

In [0]:
%sql
SELECT * FROM store_master

**Holiday Events Data Load in to Bronze Layer**

In [0]:
schema_holiday_events = spark.read.format("csv").option("inferSchema", True).option("header", "true").load(f"{rawDataSource}/holiday_events.csv").schema

In [0]:
holiday_events_df = spark.readStream.format("cloudFiles") \
  .option("cloudFiles.format", "csv") \
  .option("pathGlobFilter", "*holiday_events*.csv") \
  .option("header", True) \
  .schema(schema_holiday_events) \
  .load(f"{rawDataSource}") \
  #.withColumn("filename",input_file_name())

In [0]:
holiday_events_checkpoint_location = f"{bronzeCheckpoint}/holiday_events"
bronze_holiday_events_deltapath='/mnt/g5/bronze/holiday_events'
dbutils.fs.rm(holiday_events_checkpoint_location, True) #reset checkpoint so it reloads the file
holiday_events_df.writeStream.option("path", bronze_holiday_events_deltapath).outputMode("append").format("delta").option("checkpointLocation", holiday_events_checkpoint_location).option("mergeSchema", "true").table("holiday_events")

In [0]:
%sql
SELECT * FROM holiday_events